In [30]:
pip install python-docx

Note: you may need to restart the kernel to use updated packages.


In [31]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from docx import Document

In [32]:
def doc1(pfad):
    doc = Document(pfad)
    #Die 0 also erste Tabelle wird ausgewählt 
    table = doc.tables[0]
    #------
    #Leere Liste wird erstellt
    data = []
        #        vvv-überspringe die erste Zeile
    for i in table.rows[1:]:
        #    vvv hinten anfügen
        #             vvv entferne Leerzeichen
        #                               vvv eine Liste in der Liste welche eine Zeile (row) enthällt 
        data.append([cell.text.strip() for cell in i.cells])
        #für pandas aufbereiten als Data Frame
        #                      vvv Definierung der Spaltennamen 
    df = pd.DataFrame(data, columns=["Nr.","ÜN","Name","Vorname","Jg/AGr"," ",""," "," "," "," "," "," "," "," "," ","Su."])
    #Entfernen der überschüssigen Spalten
    df = df.drop([" "],axis = 1)
    df = df.drop(["Nr."],axis = 1)
    df = df.drop(["Jg/AGr"],axis = 1)
    df = df.drop(["Su."],axis = 1)
    #------
    #verwandelt allse strings in floats vvv fehlerhafte Werte (Sonderzeichen etc.) werden abgefangen und in NaN verwandelt
    df["ÜN"] = pd.to_numeric(df["ÜN"], errors="coerce")
    #zählt die ungültigen werte
    df["ÜN"].isna().sum()
    #gibt df zurück
    return df

In [33]:
def doc2(pfad):
    doc = Document(pfad)
    table = doc.tables[0]
    data = []
    for i in table.rows[1:]:
        data.append(
            [cell.text.strip() for cell in i.cells]
        )
    df = pd.DataFrame(
        data, columns=[
            "Nr.","ÜN","Name","Vorname","Jg/AGr"," "," "," "," "," "," "," "," "," "," "," ","Su."
        ]
    )
    df = df.drop([" "],axis = 1)
    df = df.drop(["Nr."],axis = 1)
    df = df.drop(["Jg/AGr"],axis = 1)
    df = df.drop(["Su."],axis = 1)
    
    df["ÜN"] = pd.to_numeric(df["ÜN"], errors="coerce")
    #------
    df["ÜN"].isna().sum()
    #------
    return df

In [34]:
def doc3(pfad):
    doc = Document(pfad)
    table = doc.tables[0]
    data = []
    for i in table.rows[1:]:
        data.append(
            [cell.text.strip() for cell in i.cells]
        )
    df = pd.DataFrame(
        data, columns=[
            "Nr.","ÜN","Name","Vorname","Jg/AGr"," "," "," "," "," "," "," "," "," "," "," ","Su."
        ]
    )
    df = df.drop([" "],axis = 1)
    df = df.drop(["Nr."],axis = 1)
    df = df.drop(["Jg/AGr"],axis = 1)
    df = df.drop(["Su."],axis = 1)
    
    df["ÜN"] = pd.to_numeric(df["ÜN"], errors="coerce")
    #------
    df["ÜN"].isna().sum()
    #------
    return df

In [35]:
#die Tabellen aufrufen
df_2022 = doc1(r"D:\KI_Daten\Trainings_Anwesenheit\Anwesenheit1.docx")
df_2023 = doc2(r"D:\KI_Daten\Trainings_Anwesenheit\Anwesenheit2.docx")
df_2024 = doc3(r"D:\KI_Daten\Trainings_Anwesenheit\Anwesenheit3.docx")

In [41]:
#Fügt die drei Tabellen zusammen und benennt die Spalten um
df = (
    df_2022.rename(columns={"ÜN": "ÜN_2022"})
    .merge(
        df_2023.rename(columns={"ÜN": "ÜN_2023"}),
        on=["Name", "Vorname"],
        how="outer"
    )
    .merge(
        df_2024.rename(columns={"ÜN": "ÜN_2024"}),
        on=["Name", "Vorname"],
        how="outer"
    )
)

# fehlende Werte ersetzen
df.fillna(0, inplace=True)
#zwischenstand ausgeben
df.head()
#Tabelle Speichern als csv Dokument
df.to_csv("teilnehmer_alle_jahre.csv", index=False)

In [42]:
#Schnitt erstellen
df["ÜN_2025"] = df[["ÜN_2022", "ÜN_2023", "ÜN_2024"]].mean(axis=1)

In [38]:
#Eingabe
X = df[["ÜN_2022", "ÜN_2023", "ÜN_2024"]]
#hiernach wird geprüft
y = df["ÜN_2025"]
#model initialisieren
model = LinearRegression()
#KI trainieren
model.fit(X,y)
#die fertige Prognose
df["Prognose_ÜN_"] = model.predict(X)
#Prognose Runden (für die Lesbarkeit)
df["Prognose_ÜN_Gerundet"] = df["Prognose_ÜN_"].round(0).astype(int)

In [39]:
#Ergebniss ausgeben
df.head()

,ÜN_2022,Name,Vorname,,ÜN_2023,ÜN_2024,ÜN_2025,Prognose_ÜN_,Prognose_ÜN_Gerundet
0,0.0,,,,0.0,0.0,0.000000,-1.776357e-15,0
1,30.0,Barsch,Tim,,34.0,32.0,32.000000,3.200000e+01,32
2,20.0,Blatt,Herbert,,22.0,25.0,22.333333,2.233333e+01,22
3,11.0,Esel,Lea,,16.0,10.0,12.333333,1.233333e+01,12
4,8.0,Finken,Josef,,4.0,10.0,7.333333,7.333333e+00,7


In [40]:
#Speichern
df.to_csv("teilnehmer_alle_jahre.csv", index=False)